In [5]:
genome = ''.join(line.strip() for line in open('chr1.GRCh38.excerpt.fasta') if not line.startswith('>'))

In [13]:
def editDistance(P, T):
    # Width / Height of matrix
    W = len(T) + 1
    H = len(P) + 1
    
    # Create distance matrix
    D = []
    for i in range(H):
        D.append([0]*W)
        
    # Initialize first column
    for y in range(H):
        D[y][0] = y
        
    # Initialize first row
    for x in range(W):
        D[0][x] = 0
        
    # Fill in the rest of the matrix
    for y in range(1, H):
        for x in range(1, W):
            distHor = D[y][x-1] + 1
            distVer = D[y-1][x] + 1
            if P[y-1] == T[x-1]:
                distDiag = D[y-1][x-1]
            else:
                distDiag = D[y-1][x-1] + 1
            D[y][x] = min(distHor, distVer, distDiag)
            
    # Edit distance is the value in the bottom right corner of the matrix
    return min(D[-1])

In [14]:
P = 'GCGTATGC'
T = 'TATTGGCTATACGGTT'
print editDistance(P,T)

2


In [16]:
# Question 1
print editDistance('GCTGATCGATCGTACG', genome)

3


In [17]:
# Question 2
print editDistance('GATTTACCAGATTGAG', genome)

2


In [18]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [21]:
def charToQual(c):
    return ord(c) - 33

def loadFastq(filename):
    reads = []
    quals = []
    with open(filename) as f:
        while True:
            id_line = f.readline().rstrip()
            if not id_line:
                break
            assert id_line.startswith('@')
            reads.append(f.readline().rstrip())
            assert f.readline().rstrip() == '+'
            quals.append(map(charToQual, f.readline().rstrip()))
            assert len(reads[-1]) == len(quals[-1])
    return reads, quals

In [22]:
reads = loadFastq('ERR266411_1.for_asm.fastq')[0]

In [31]:
from collections import defaultdict

def buildKmerMap(reads, n):
    m = defaultdict(set)
    for read in reads:
        for i in range(len(read) - n + 1):
            kmer = read[i:i+n]
            m[kmer].add(read)
    return m

In [32]:
m = buildKmerMap(reads, 30)

In [33]:
len(m)

108344

In [39]:
def buildOverlapGraph(reads, n):
    kmerMap = buildKmerMap(reads, n)
    overlaps = defaultdict(set)
    for read in reads:
        suffix = read[-n:]
        candidates = kmerMap[suffix]
        for other_read in candidates:
            if other_read is read:
                continue
            overlap_len = overlap(read, other_read, n)
            if overlap_len > 0:
                overlaps[read].add((other_read, overlap_len))
    return overlaps

In [40]:
# Check: http://nbviewer.ipython.org/github/BenLangmead/ads1-hw-examples/blob/master/hw3_overlap_all.ipynb
buildOverlapGraph(['ABCDEFG', 'EFGHIJ', 'HIJABC'], 3)

defaultdict(set,
            {'ABCDEFG': {('EFGHIJ', 3)},
             'EFGHIJ': {('HIJABC', 3)},
             'HIJABC': {('ABCDEFG', 3)}})

In [41]:
buildOverlapGraph(['ABCDEFG', 'EFGHIJ', 'HIJABC'], 4)

defaultdict(set, {})

In [42]:
buildOverlapGraph(['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT'], 4)

defaultdict(set,
            {'ACGTAC': {('CGTACG', 5), ('GTACGA', 4), ('GTACGT', 4)},
             'CGTACG': {('GTACGA', 5),
              ('GTACGT', 5),
              ('TACGAT', 4),
              ('TACGTA', 4)},
             'GTACGA': {('TACGAT', 5)},
             'GTACGT': {('ACGTAC', 4), ('TACGTA', 5)},
             'TACGTA': {('ACGTAC', 5), ('CGTACG', 4)}})

In [43]:
# Question 3
g = buildOverlapGraph(reads, 30)

In [46]:
sum(len(v) for (k,v) in g.iteritems())

904746

In [47]:
len(g)

7161